In [37]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

import pandas as pd
import numpy as np
import json

In [38]:
def load_df(file_name : str, num_users : int):
    path = file_name + '.csv'
    df = pd.read_csv(path)
    new_col_name = str(num_users)+'_usr'
    #df.set_index('Name', inplace=True, drop=False)
    df.drop(df.tail(1).index , inplace = True)
    df.rename(columns = {'Average Response Time':new_col_name}, inplace = True)
    #df = df[[new_col_name]].apply(lambda x : np.log(x))
    #df.index=np.log(df.index)
    df = df[[new_col_name]].apply(lambda x : x)
    df = df.iloc[:150]

    return df


In [39]:
el_usr1 = load_df('results/EL_usr1_spwr1_st0', 1)

el_usr2 = load_df('results/EL_usr2_spwr1_st0', 2)

merged_pd = pd.concat(
    [
        el_usr1, 
        el_usr2
    ], axis = 1)

el_usr5 = load_df('results/EL_usr5_spwr1_st0', 5)
merged_pd = pd.concat(
    [
        merged_pd,
        el_usr5
    ], axis = 1)

el_usr10 = load_df('results/EL_usr10_spwr1_st0', 10)
merged_pd = pd.concat(
    [
        merged_pd,
        el_usr10
    ], axis = 1)

el_usr20 = load_df('results/EL_usr20_spwr1_st0', 20)
merged_pd = pd.concat(
    [
        merged_pd,
        el_usr20
    ], axis = 1)

el_usr50 = load_df('results/EL_usr50_spwr1_st0', 50)
merged_pd = pd.concat(
    [
        merged_pd,
        el_usr50
    ], axis = 1)

In [40]:
figure1 = px.line(data_frame=merged_pd, 
                       color_discrete_sequence=px.colors.qualitative.G10)

figure2 = px.box(data_frame=merged_pd,
                 x =['1_usr', '2_usr', '5_usr', '10_usr', '20_usr', '50_usr'],
                 color_discrete_sequence=px.colors.qualitative.G10[6:])

# For as many traces that exist per Express figure, get the traces from each plot and store them in an array.
# This is essentially breaking down the Express fig into it's traces
figure1_traces = []
figure2_traces = []

for trace in range(len(figure1["data"])):
    figure1_traces.append(figure1["data"][trace])
for trace in range(len(figure2["data"])):
    figure2_traces.append(figure2["data"][trace])

#Create a 1x2 subplot
aggregated_results = make_subplots(rows=2, cols=1, 
                                   vertical_spacing=0.15,
                                   subplot_titles=['Average Response Time', 'Average Response Time Boxplot']) 

# Get the Express fig broken down as traces and add the traces to the proper plot within in the subplot
for traces in figure1_traces:
    aggregated_results.append_trace(traces, row=1, col=1)
for traces in figure2_traces:
    aggregated_results.append_trace(traces, row=2, col=1)


aggregated_results.update_xaxes(title_text="Execution Time", row=1, col=1)
aggregated_results.update_yaxes(title_text="Milliseconds (ms)", row=1, col=1)

aggregated_results.update_xaxes(title_text="Milliseconds (ms)", row=2, col=1)
aggregated_results.update_yaxes(title_text="Number of users", row=2, col=1)

aggregated_results.update_layout(height=900, width=1024)
aggregated_results.show()

In [41]:
aggregated_results.write_image('plots/aggregated_results.pdf', width=1024, validate = True)

In [42]:
figure1.update_xaxes(title_text="Execution Time")
figure1.update_yaxes(title_text="Milliseconds (ms)")
figure1.write_image('plots/aggregated_lineplot.pdf', width=1024, validate = True)

In [43]:
figure2.update_xaxes(title_text="Milliseconds (ms)")
figure2.update_yaxes(title_text="Number of users")
figure2.write_image('plots/aggregated_boxplot.pdf', width=1024, validate = True)

# Plot results 50 users slow spawning

In [33]:
def locust_stats_parser(file_path : str):
    stats = []

    with open(file_path, 'r') as fp:
        for line in fp:
            if "total_rps" in line:
                total_rps = line
                user_count = next(fp)

                jsonString = '{'+total_rps+user_count.replace(',', '')+'}'
                stats.append(json.loads(jsonString))

    return stats

In [34]:
data = locust_stats_parser('stats-50usrs_slow.json')
dataframe = pd.DataFrame(data)

In [35]:
window_size = 30
dataframe['rps_moving_average'] = dataframe['total_rps'].rolling(window_size,axis = 0).mean().fillna(method='backfill')

In [36]:

figure1 = px.line(data_frame=dataframe, 
                  y=['total_rps', 'rps_moving_average'],
                  color_discrete_sequence=px.colors.qualitative.G10[0:2])


figure2 = px.line(data_frame=dataframe,
                  y = ['user_count'], 
                  color_discrete_sequence=px.colors.qualitative.G10[3:4])


indexes = np.arange(0, dataframe['user_count'].shape[0], step=20)
dashed_line = np.array([35]*indexes.shape[0]) 

figure3 = go.Scattergl(name = 'user_count=35', x = indexes, y=dashed_line, line_dash='dash')

# For as many traces that exist per Express figure, get the traces from each plot and store them in an array.
# This is essentially breaking down the Express fig into it's traces
figure1_traces = []
figure2_traces = []

for trace in range(len(figure1["data"])):
    figure1_traces.append(figure1["data"][trace])

for trace in range(len(figure2["data"])):
    figure2_traces.append(figure2["data"][trace])

#Create a 1x2 subplot
rps_50slow = make_subplots(rows=2, cols=1,
                           shared_xaxes='columns',
                           vertical_spacing=0.15,
                           x_title= 'Execution Time',
                           row_heights=[0.7, 0.3],
                           subplot_titles=['Requests fulfilled per seconds', 'Number of users']) 

# Get the Express fig broken down as traces and add the traces to the proper plot within in the subplot
for traces in figure1_traces:
    rps_50slow.append_trace(traces, row=1, col=1)
for traces in figure2_traces:
    rps_50slow.append_trace(traces, row=2, col=1)

rps_50slow.append_trace(figure3, row=2, col=1)

rps_50slow.update_yaxes(range=[0, 1.7], row=1, col=1)
rps_50slow.update_yaxes(range=[0, 60], row=2, col=1)

rps_50slow.update_layout(height=700, width=1024)
rps_50slow.show()

In [14]:
rps_50slow.write_image(file = 'plots/rps_50slow.pdf', format = 'pdf', width=1024, validate=True)